<a href="https://colab.research.google.com/github/nikotang/RD-UU-MPLT/blob/main/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R&D

roberta large is only a bit better than base in mnli, large finetuned on mnli did no better than just large (https://github.com/facebookresearch/fairseq/tree/main/examples/roberta)

finetuning roberta on winogrande (https://github.com/facebookresearch/fairseq/blob/main/examples/roberta/wsc/README.md)

extract mnli and winogrande test sets

scramble test sets + ways to evaluate scrambling (e.g. scoring alignments?)

finetuning roberta large with transformed winogrande and original winogrande to see the effects of the transform (if bad, try sharing layers/multitask learning?)

do setup 1


get test set

# 0: Installations and imports

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import pickle
# import os
import numpy as np

from transformers import AutoTokenizer, RobertaForSequenceClassification, EarlyStoppingCallback
from datasets import Dataset, load_dataset, concatenate_datasets, DatasetDict
import pandas as pd

# 1: Preprocessing data

## Multi-NLI

In [ ]:
mnli_dataset = load_dataset('multi_nli')      # load from Huggingface dataset hub

# remove irrelevant columns
mnli_dataset = mnli_dataset.remove_columns(['promptID', 'pairID', 'premise_binary_parse', 'premise_parse', \
                                            'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])
# rename some columns
mnli_dataset = mnli_dataset.rename_column('premise', 'sentence1')
mnli_dataset = mnli_dataset.rename_column('hypothesis', 'sentence2')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# create a test set

matched = mnli_dataset['validation_matched'].train_test_split(test_size=0.5)
mismatched = mnli_dataset['validation_mismatched'].train_test_split(test_size=0.5)

del mnli_dataset['validation_matched']
del mnli_dataset['validation_mismatched']

mnli_dataset['validation'] = concatenate_datasets([matched['train'], mismatched['train']])
mnli_dataset['test'] = concatenate_datasets([matched['test'], mismatched['test']])

In [ ]:
# relabel the dataset

label2id = {"contradiction": 0, "neutral": 0, "entailment": 1}

mnli_dataset = mnli_dataset.align_labels_with_mapping(label2id, "label")

Aligning the labels:   0%|          | 0/10 [00:00<?, ?ba/s]

Aligning the labels:   0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
# trim training set
mnli_dataset['train'] = Dataset.from_dict(mnli_dataset['train'][:100000])

## PAWS

In [ ]:
paws_dataset = load_dataset("paws", "labeled_final")
paws2 = load_dataset("paws", "labeled_swap")

paws_dataset['train'] = concatenate_datasets([paws_dataset['train'], paws2['train']])

paws_dataset = paws_dataset.remove_columns('id')

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Winogrande

In [ ]:
!wget -N https://raw.githubusercontent.com/nikotang/RD-UU-MPLT/main/winogrande_1.1/train_xl_transformed.tsv
df_train = pd.read_csv('train_xl_transformed.tsv', sep='\t')
winogrande_train = Dataset.from_pandas(df_train)
winogrande_dataset = winogrande_train.train_test_split(test_size=0.1)

!wget -N https://raw.githubusercontent.com/nikotang/RD-UU-MPLT/main/winogrande_1.1/dev_transformed.tsv
df_dev = pd.read_csv('dev_transformed.tsv', sep='\t')
winogrande_dataset['validation'] = Dataset.from_pandas(df_dev)
winogrande_dataset = winogrande_dataset.remove_columns('id')

--2022-12-06 23:39:02--  https://raw.githubusercontent.com/nikotang/RD-UU-MPLT/main/winogrande_1.1/dev_transformed.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 547830 (535K) [text/plain]
Saving to: ‘dev_transformed.tsv’

dev_transformed.tsv 100%[===================>] 534.99K  --.-KB/s    in 0.01s   

Last-modified header missing -- time-stamps turned off.
2022-12-06 23:39:02 (41.9 MB/s) - ‘dev_transformed.tsv’ saved [547830/547830]



## HANS

In [ ]:
hans_dataset = load_dataset("hans")


Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset hans downloaded and prepared to /root/.cache/huggingface/datasets/hans/plain_text/1.0.0/452e93cf5383f5ae39088254215b517d0da98ccaaf0af8f7ab04d8f23f67dbd9. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
hans_dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'heuristic'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'heuristic'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'heuristic'],
        num_rows: 15000
    })
})

In [ ]:
hans_dataset = hans_dataset.remove_columns(['parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'subcase', 'template'])

# rename some columns
hans_dataset = hans_dataset.rename_column('premise', 'sentence1')
hans_dataset = hans_dataset.rename_column('hypothesis', 'sentence2')

In [ ]:
val_test = hans_dataset['validation'].train_test_split(test_size=0.5) # shuffle default=True

hans_dataset['validation'] = val_test['train']
hans_dataset['test'] = val_test['test']

In [ ]:
for h in ('lexical_overlap', 'subsequence', 'constituent'):
  print(f"{h}: {hans_dataset['validation']['heuristic'].count(h)}")

lexical_overlap: 5012
subsequence: 4929
constituent: 5059


In [ ]:
# relabel the dataset

label2id = {"non-entailment": 0, "entailment": 1}

hans_dataset = hans_dataset.align_labels_with_mapping(label2id, "label")

Aligning the labels:   0%|          | 0/30 [00:00<?, ?ba/s]

Aligning the labels:   0%|          | 0/15 [00:00<?, ?ba/s]

Aligning the labels:   0%|          | 0/15 [00:00<?, ?ba/s]

## Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize(ds):
  return tokenizer(ds['sentence1'], ds['sentence2'], padding=True, return_tensors="pt")

def process(dataset_d):
  return dataset_d.map(tokenize, batched=True, batch_size=64, num_proc=4)

# tokenize datasets
mnli_dataset_tok = process(mnli_dataset)
paws_dataset_tok = process(paws_dataset)
winogrande_dataset_tok = process(winogrande_dataset)
hans_dataset_tok = process(hans_dataset)

# rename 'label' to 'labels' for Trainer (alternatively specify 'label' in Trainer params)
mnli_dataset_tok = mnli_dataset_tok.rename_column('label', 'labels')
paws_dataset_tok = paws_dataset_tok.rename_column('label', 'labels')
winogrande_dataset_tok = winogrande_dataset_tok.rename_column('label', 'labels')
hans_dataset_tok = hans_dataset_tok.rename_column('label', 'labels')

for dataset_d in [mnli_dataset_tok, paws_dataset_tok, winogrande_dataset_tok, hans_dataset_tok]:
  dataset_d.set_format(type='torch', columns=['labels'], output_all_columns=True)
hans_dataset_tok.set_format(type='torch', columns=['labels'], output_all_columns=True)

#0:   0%|          | 0/118 [00:00<?, ?ba/s]

#1:   0%|          | 0/118 [00:00<?, ?ba/s]

#2:   0%|          | 0/118 [00:00<?, ?ba/s]

#3:   0%|          | 0/118 [00:00<?, ?ba/s]

#2:   0%|          | 0/59 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/59 [00:00<?, ?ba/s]

#1:   0%|          | 0/59 [00:00<?, ?ba/s]

#0:   0%|          | 0/59 [00:00<?, ?ba/s]

#2:   0%|          | 0/59 [00:00<?, ?ba/s]

#3:   0%|          | 0/59 [00:00<?, ?ba/s]

In [ ]:
mnli_dataset_tok.save_to_disk('./mnli_datasets')
paws_dataset_tok.save_to_disk('./paws_datasets')
winogrande_dataset_tok.save_to_disk('./winogrande_datasets')
hans_dataset_tok.save_to_disk('./hans_datasets')


Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15000 [00:00<?, ? examples/s]

In [ ]:
!zip -r mnli.zip mnli_datasets
!zip -r paws.zip paws_datasets
!zip -r winogrande.zip winogrande_datasets
!zip -r hans.zip hans_datasets

  adding: hans_datasets/ (stored 0%)
  adding: hans_datasets/train/ (stored 0%)
  adding: hans_datasets/train/dataset_info.json (deflated 57%)
  adding: hans_datasets/train/data-00000-of-00001.arrow (deflated 87%)
  adding: hans_datasets/train/state.json (deflated 38%)
  adding: hans_datasets/dataset_dict.json (deflated 5%)
  adding: hans_datasets/validation/ (stored 0%)
  adding: hans_datasets/validation/dataset_info.json (deflated 57%)
  adding: hans_datasets/validation/data-00000-of-00001.arrow (deflated 93%)
  adding: hans_datasets/validation/state.json (deflated 38%)
  adding: hans_datasets/test/ (stored 0%)
  adding: hans_datasets/test/dataset_info.json (deflated 57%)
  adding: hans_datasets/test/data-00000-of-00001.arrow (deflated 93%)
  adding: hans_datasets/test/state.json (deflated 38%)


# OR 1.5 load already preprocessed data from here

In [ ]:
# !wget -N https://github.com/nikotang/RD-UU-MPLT/raw/main/mnli.zip
# !wget -N https://github.com/nikotang/RD-UU-MPLT/raw/main/paws.zip
# !wget -N https://github.com/nikotang/RD-UU-MPLT/raw/main/winogrande.zip

# !unzip mnli.zip
# !unzip paws.zip
# !unzip winogrande.zip

from datasets import load_from_disk

mnli_dataset_tok = load_from_disk('./mnli_datasets')
paws_dataset_tok = load_from_disk('./paws_datasets')
winogrande_dataset_tok = load_from_disk('./winogrande_datasets')



In [ ]:
mnli_dataset_tok['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
paws_dataset_tok['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
print(paws_dataset_tok['train']['labels'][:10])

tensor([0, 1, 0, 1, 1, 1, 0, 1, 0, 0])


In [ ]:
paws_dataset_tok = paws_dataset_tok.cast_column('labels', Value(dtype='int64'))

Casting the dataset:   0%|          | 0/80 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
print(paws_dataset_tok['train']['labels'][:10])

tensor([0, 1, 0, 1, 1, 1, 0, 1, 0, 0])


In [ ]:
from datasets import Features, Value

features = Features({'labels': Value(dtype='int64')})

features

{'labels': Value(dtype='int64', id=None)}

In [ ]:
mnli_l = mnli_dataset_tok['train']['labels']
paws_l = paws_dataset_tok['train']['labels']

mnli_dataset_tok = mnli_dataset_tok.remove_columns(['labels'])
paws_dataset_tok = paws_dataset_tok.remove_columns(['labels'])

In [ ]:
mnli_paws_dataset_t = concatenate_datasets([mnli_dataset_tok['train'], paws_dataset_tok['train']])

In [ ]:
new = torch.cat((mnli_l, paws_l), -1)


In [ ]:
paws_l.shape

torch.Size([79798])

In [ ]:
new.shape

torch.Size([179798])

In [ ]:
mnli_paws_dataset_t['labels'] = new

TypeError: ignored

In [ ]:
# mini sets for toying

mnli_mini = Dataset.from_dict(mnli_dataset_tok['train'][:5000])
paws_mini = Dataset.from_dict(paws_dataset_tok['train'][:5000])
winogrande_mini = Dataset.from_dict(winogrande_dataset_tok['train'][:5000])

mnli_val_mini = Dataset.from_dict(mnli_dataset_tok['validation'][:2000])


In [ ]:
# a tokenized example

tokenizer.decode(mnli_mini[900]['input_ids'])


'<s>Although merger discussions continued among the four programs, in March 2000 two of the programs did not vote in favor of the merger plan, and therefore the plan moved forward with only two  Legal Services Organization of Indiana, Inc. and Legal Services of Northwest Indiana, Inc.</s></s>Among the four programs, merger discussions continued and 2 of the programs did not vote in march 2000.</s><pad><pad>'

# 2: Training

In [ ]:
# https://huggingface.co/docs/transformers/training
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
dataset = mnli_dataset_tok

name = 'mnli'

from transformers import Trainer, TrainingArguments
import gc

gc.collect()
torch.cuda.empty_cache()

CUDA_VISIBLE_DEVICES=0

training_args = TrainingArguments(
    output_dir=f'./{name}_results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=1000,                # number of warmup steps for learning rate scheduler
    learning_rate=5e-5,
    weight_decay=5e-4,               # strength of weight decay
    logging_dir=f'./{name}_logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy="steps",      # or 'steps', then specify no. of 'eval_steps'
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'      # determine 'best' according to val acc
)

model = RobertaForSequenceClassification.from_pretrained('roberta-base').to("cuda")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=mnli_mini,         # training dataset
    eval_dataset=mnli_val_mini,             # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]      # checks 3 more steps before early stopping
)

trainer.train()

trainer.save_model()

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Step,Training Loss,Validation Loss,Accuracy
200,0.538700,0.525822,0.743000
400,0.427000,0.628128,0.764500
600,0.348400,0.394730,0.848500
800,0.250900,0.426941,0.860500


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to ./mnli_results/checkpoint-200
Configuration saved in ./mnli_results/checkpoint-200/config.json
Model weights saved in ./mnli_results/checkpoint-200/pytorch_model.bin
tokenizer config file saved in ./mnli_results/checkpoint-200/tokenizer_config.json
Special tokens file saved in ./mnli_results/checkpoint-200/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely

# 3: Evaluate

In [ ]:
from sys import argv

assert len(argv) == 3
MODEL_DIR = argv[1]
OUTPUT_DIR = argv[2]


test_model = RobertaForSequenceClassification.from_pretrained(MODEL_DIR).to("cuda")
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

test_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 64
)

tester = Trainer(
              model = test_model,
              args = test_args,
              compute_metrics = compute_metrics)


In [ ]:
# get test set accuracy
trainer.evaluate(eval_dataset=mnli_dataset_tok['test'])


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9824
  Batch size = 64


{'eval_loss': 0.4031834900379181,
 'eval_accuracy': 0.8722516286644951,
 'eval_runtime': 61.6535,
 'eval_samples_per_second': 159.342,
 'eval_steps_per_second': 2.498,
 'epoch': 3.0}

In [ ]:
logits, references, _ = trainer.predict(mnli_dataset_tok['test'])


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 9824
  Batch size = 64


In [ ]:
from statistics import mean
from collections import Counter

softmax = nn.Softmax(dim=-1)

soft_logits = [softmax(torch.tensor(logit)) for logit in logits]
soft_logits = np.stack(np.array(soft_logits, dtype=object))

preds = np.argmax(soft_logits, axis=1)
pred_probs = np.amax(soft_logits, axis=1)

# save the results
results = np.concatenate((pred_probs, preds, references), axis=1)
with open('results.npy', 'wb') as f:
  np.save(f, results)

# to load the file:
# with open('results.npy', 'rb') as f:
#   results = np.load(f)

In [ ]:
avg_confidence = np.mean(pred_probs)
print(avg_confidence)

confidence_1 = []
confidence_0 = []
for i, prob in enumerate(pred_probs):
  if preds[i] == 1:
    confidence_1.append(prob)
  else:
    confidence_0.append(prob)

print(Counter(preds))
print(mean(confidence_0))
print(mean(confidence_1))


<ipython-input-61-d34e29e7bb43>:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  soft_logits = np.stack(np.array(soft_logits, dtype=object))


0.96136194
Counter({0: 6424, 1: 3400})
0.96917564
0.9465988
